In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

from unlearning.tool import get_hf_model
from unlearning.feature_activation import get_forget_retain_data, tokenize_dataset, get_mean_feature_activation, get_top_features
from unlearning.jump_relu import load_gemma2_2b_sae
from unlearning.intervention import scaling_intervention
from unlearning.metrics import calculate_MCQ_metrics

In [2]:
model = get_hf_model('google/gemma-2-2b-it')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Save top features for each layer

In [3]:
forget_dataset, retain_dataset = get_forget_retain_data('bio-forget-corpus', 'wikitext')

print(len(forget_dataset), len(forget_dataset[0]))
print(len(retain_dataset), len(retain_dataset[0]))

forget_tokens = tokenize_dataset(model, forget_dataset)
retain_tokens = tokenize_dataset(model, retain_dataset)

print(forget_tokens.shape, retain_tokens.shape)

24432 16027
1962 859
torch.Size([153108, 1024]) torch.Size([275, 1024])


In [10]:
LAYER = 1
sae = load_gemma2_2b_sae(layer=LAYER)


Found SAE with l0=102 at path google/gemma-scope-2b-pt-res/layer_1/width_16k/average_l0_102/params.npz


params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [24]:
def save_sae_top_features(model, layer, forget_tokens, retain_tokens):
    sae = load_gemma2_2b_sae(layer=layer)

    # shuffle forget_tokens 
    shuffled_forget_tokens = forget_tokens[torch.randperm(forget_tokens.shape[0])]

    mean_feature_activation_forget = get_mean_feature_activation(model, sae, shuffled_forget_tokens[:2048], batch_size=8)
    mean_feature_activation_retain = get_mean_feature_activation(model, sae, retain_tokens, batch_size=8)

    top_features = get_top_features(mean_feature_activation_forget, mean_feature_activation_retain, 0.01)
    np.savetxt(f'../data/top_features/gemma-2-2b-it/layer{layer}.txt', top_features, fmt='%d')

In [25]:
for layer in [0] + list(range(4, model.config.num_hidden_layers)):
    print(f'Layer {layer}')
    save_sae_top_features(model, layer, forget_tokens, retain_tokens)

Layer 0
Found SAE with l0=105 at path google/gemma-scope-2b-pt-res/layer_0/width_16k/average_l0_105/params.npz


params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

100%|██████████| 35/35 [00:31<00:00,  1.11it/s]


[ 3148  7146   611  2261  3873 13553 13954  7076   443   721  5531 13030
 13361  7126  6208  5361  5158  1196 15250  7510]
Layer 4
Found SAE with l0=124 at path google/gemma-scope-2b-pt-res/layer_4/width_16k/average_l0_124/params.npz


params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

...

100%|██████████| 35/35 [00:32<00:00,  1.09it/s]


[ 3867 12292 16337 10961  9233  6139  5490  5643 13830  6637  9021 10117
 13129  8925 16168  4969  3520  6690  7967 14983]
Layer 25
Found SAE with l0=116 at path google/gemma-scope-2b-pt-res/layer_25/width_16k/average_l0_116/params.npz


params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

100%|██████████| 35/35 [00:32<00:00,  1.09it/s]

[14035  3685 12093  9930  9811  3016  9501   335  1915 11512 13413  1674
 16192  6926 14798 14634  8620 14949  8850 15033]


In [13]:
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

get_neuronpedia_quick_list(top_features[:20], layer=LAYER, model='gemma-2-2b', dataset='gemmascope-res-16k')

'https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%226688%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%2214468%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%22691%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%226146%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%224825%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%2215322%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%221-gemmascope-res-16k%22%2C%20%22index%22%3A%20%2210572%22%7D%2C%20%7B%22modelId%22%3A%20%22g

### try ablate multilayer sae features

In [26]:
saes = {layer: load_gemma2_2b_sae(layer=layer) for layer in range(model.config.num_hidden_layers)}

top_features_all_layers = {
    layer: np.loadtxt(f'../data/top_features/gemma-2-2b-it/layer{layer}.txt', dtype=int) 
    for layer in range(model.config.num_hidden_layers)
}

Found SAE with l0=105 at path google/gemma-scope-2b-pt-res/layer_0/width_16k/average_l0_105/params.npz
Found SAE with l0=102 at path google/gemma-scope-2b-pt-res/layer_1/width_16k/average_l0_102/params.npz
Found SAE with l0=141 at path google/gemma-scope-2b-pt-res/layer_2/width_16k/average_l0_141/params.npz
Found SAE with l0=59 at path google/gemma-scope-2b-pt-res/layer_3/width_16k/average_l0_59/params.npz
Found SAE with l0=124 at path google/gemma-scope-2b-pt-res/layer_4/width_16k/average_l0_124/params.npz
Found SAE with l0=68 at path google/gemma-scope-2b-pt-res/layer_5/width_16k/average_l0_68/params.npz
Found SAE with l0=70 at path google/gemma-scope-2b-pt-res/layer_6/width_16k/average_l0_70/params.npz
Found SAE with l0=69 at path google/gemma-scope-2b-pt-res/layer_7/width_16k/average_l0_69/params.npz
Found SAE with l0=71 at path google/gemma-scope-2b-pt-res/layer_8/width_16k/average_l0_71/params.npz
Found SAE with l0=73 at path google/gemma-scope-2b-pt-res/layer_9/width_16k/average

In [29]:
top_features_all_layers[0]

array([ 3148,  7146,   611, ..., 15450,  4960, 12182])

In [47]:
from dataclasses import dataclass

@dataclass
class AblationResults:
    multiplier: int
    n_features: int
    acc: dict[str, float]
    full_metrics: dict[str, dict]

In [49]:
from contextlib import ExitStack

def run_interventions(model, saes, top_features, n_features, multiplier, datasets):
    with ExitStack() as stack:
        for layer in range(model.config.num_hidden_layers):
            stack.enter_context(scaling_intervention(
                model, layer, saes[layer], top_features[layer][:n_features], multiplier
            ))

        acc = {}
        full_metrics = {}
        for dataset_name in datasets:
            metrics = calculate_MCQ_metrics(model, dataset_name=dataset_name)
            acc[dataset_name] = metrics['mean_correct']
            full_metrics[dataset_name] = metrics
        
    print(f'{n_features} features, {multiplier}x')
    print(acc)
    return AblationResults(multiplier=multiplier, n_features=n_features, acc=acc, full_metrics=full_metrics)



In [50]:
n_features = 20
multiplier = 20
datasets = ['wmdp-bio', 'high_school_us_history', 'high_school_geography', 'college_computer_science', 'human_aging', 'college_biology']

all_results = []
for n_features in [10, 20, 50, 100]:
    for multiplier in [1, 5, 10, 20, 50, 100]:
        result = run_interventions(model, saes, top_features_all_layers, n_features, multiplier, datasets)
        all_results.append(result)


100%|██████████| 24/24 [00:10<00:00,  2.28it/s]


10 features, 1x
{'wmdp-bio': 0.604084849357605, 'high_school_us_history': 0.7352941632270813, 'high_school_geography': 0.752525269985199, 'college_computer_science': 0.3999999761581421, 'human_aging': 0.6322870254516602, 'college_biology': 0.6597222089767456}


100%|██████████| 24/24 [00:10<00:00,  2.26it/s]


10 features, 5x
{'wmdp-bio': 0.5027494430541992, 'high_school_us_history': 0.7107843160629272, 'high_school_geography': 0.7424242496490479, 'college_computer_science': 0.4099999964237213, 'human_aging': 0.6098654866218567, 'college_biology': 0.5833333134651184}


100%|██████████| 24/24 [00:10<00:00,  2.26it/s]


...

100 features, 20x
{'wmdp-bio': 0.23644933104515076, 'high_school_us_history': 0.27941176295280457, 'high_school_geography': 0.2222222238779068, 'college_computer_science': 0.25, 'human_aging': 0.30941706895828247, 'college_biology': 0.2916666567325592}


100%|██████████| 24/24 [00:10<00:00,  2.25it/s]


100 features, 50x
{'wmdp-bio': 0.2702278196811676, 'high_school_us_history': 0.2647058963775635, 'high_school_geography': 0.21717171370983124, 'college_computer_science': 0.25, 'human_aging': 0.286995530128479, 'college_biology': 0.2083333283662796}


100%|██████████| 24/24 [00:10<00:00,  2.25it/s]

100 features, 100x
{'wmdp-bio': 0.24666143953800201, 'high_school_us_history': 0.25, 'high_school_geography': 0.1818181872367859, 'college_computer_science': 0.25999999046325684, 'human_aging': 0.30941706895828247, 'college_biology': 0.2569444477558136}


In [37]:
baseline_results

{'wmdp-bio': 0.6355066895484924,
 'high_school_us_history': 0.7401961088180542,
 'high_school_geography': 0.7575757503509521,
 'college_computer_science': 0.44999998807907104,
 'human_aging': 0.6322870254516602,
 'college_biology': 0.7083333134651184}

In [35]:
results # 20 features each layer, zero ablate

{'wmdp-bio': 0.6245090365409851,
 'high_school_us_history': 0.7450980544090271,
 'high_school_geography': 0.752525269985199,
 'college_computer_science': 0.44999998807907104,
 'human_aging': 0.6233184337615967,
 'college_biology': 0.6875}

In [39]:
zero_ablate_results # zero ablate 40 features

{'wmdp-bio': 0.628436803817749,
 'high_school_us_history': 0.7352941632270813,
 'high_school_geography': 0.7575757503509521,
 'college_computer_science': 0.4399999976158142,
 'human_aging': 0.6367713212966919,
 'college_biology': 0.7013888955116272}

In [43]:
zero_ablate_results_1000 # zero ablate 1000 features

{'wmdp-bio': 0.6111547350883484,
 'high_school_us_history': 0.7401961088180542,
 'high_school_geography': 0.6616161465644836,
 'college_computer_science': 0.3999999761581421,
 'human_aging': 0.573991060256958,
 'college_biology': 0.6527777910232544}

In [45]:
zero_ablate_results_5000 # zero ablate 1000 features

{'wmdp-bio': 0.2820110023021698,
 'high_school_us_history': 0.30882352590560913,
 'high_school_geography': 0.2626262605190277,
 'college_computer_science': 0.25,
 'human_aging': 0.31838566064834595,
 'college_biology': 0.2708333432674408}

In [33]:
results # 20 features each layer, multiplier 20

{'wmdp-bio': 0.3244304955005646,
 'high_school_us_history': 0.7352941632270813,
 'high_school_geography': 0.7272727489471436,
 'college_computer_science': 0.41999998688697815,
 'human_aging': 0.5291479825973511,
 'college_biology': 0.4513888955116272}

In [18]:


# layer = 3
# features_to_ablate = list(top_features[:80])
# multiplier = 20

n_features = 20
multiplier = 20
with scaling_intervention(model, 1, saes[1], top_features[1][:n_features], multiplier):
    with scaling_intervention(model, 2, saes[2], top_features[2][:n_features], multiplier):
        with scaling_intervention(model, 3, saes[3], top_features[3][:n_features], multiplier):

            intervened_metrics = calculate_MCQ_metrics(model)
            intervened_history_metrics = calculate_MCQ_metrics(model, dataset_name='high_school_us_history')
            interved_human_aging_metrics = calculate_MCQ_metrics(model, dataset_name='human_aging')
            intervened_college_bio_metrics = calculate_MCQ_metrics(model, dataset_name='college_biology')
            
            print(f"n_features: {n_features}, multiplier: {multiplier}")
            print(f"\t\twmdp-bio: {intervened_metrics['mean_correct']}")
            print(f"\t\thigh_school_us_history: {intervened_history_metrics['mean_correct']}")
            print(f"\t\thuman_aging: {interved_human_aging_metrics['mean_correct']}")
            print(f"\t\tcollege_bio: {intervened_college_bio_metrics['mean_correct']}")
    

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

100%|██████████| 213/213 [00:26<00:00,  7.93it/s]


Generating test split:   0%|          | 0/204 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/22 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/144 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

100%|██████████| 24/24 [00:03<00:00,  7.87it/s]

n_features: 20, multiplier: 20
		wmdp-bio: 0.3244304955005646
		high_school_us_history: 0.7352941632270813
		human_aging: 0.5291479825973511
		college_bio: 0.4513888955116272


In [16]:
top_features[1].shape

(8820,)